## References

* [MNIST](http://yann.lecun.com/exdb/mnist/)
* [Deep MNIST for experts](https://www.tensorflow.org/get_started/mnist/pros)

In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.examples.tutorials.mnist.input_data import read_data_sets as load_data

np.random.seed(0)
tf.set_random_seed(0)

In [ ]:
data = load_data('data', one_hot=True)

print('Training: {}'.format(data.train.num_examples))
print('Validation: {}'.format(data.validation.num_examples))
print('Testing: {}'.format(data.test.num_examples))

In [ ]:
tf.reset_default_graph()

class Model:
    def __init__(self):
        self.x = tf.placeholder(tf.float32, shape=[None, 784])
        self.y = tf.placeholder(tf.float32, shape=[None, 10])
        W = tf.Variable(tf.zeros([784, 10]))
        b = tf.Variable(tf.zeros([10]))
        self.y_logit = tf.matmul(self.x, W) + b

class Objective:
    def __init__(self, model, **arguments):
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            labels=model.y, logits=model.y_logit))
        optimizer = tf.train.GradientDescentOptimizer(**arguments)
        self.train = optimizer.minimize(self.loss)

class Experiment:
    def run(self, data, model, objective, session, batch_size=32, step_count=1000):
        for _ in range(step_count):
            batch = data.train.next_batch(batch_size)
            feed = {model.x: batch[0], model.y: batch[1]}
            session.run(objective.train, feed_dict=feed)
        feed = {model.x: data.test.images, model.y: data.test.labels}
        y_score = session.run(tf.nn.softmax(model.y_logit), feed_dict=feed)
        y_predicted = np.argmax(y_score, axis=1)
        y_test = np.argmax(data.test.labels, axis=1)
        return self._assess(y_test, y_predicted, y_score)

    def _assess(self, y_test, y_predicted, y_score):
        total = len(y_test)
        return {
            'Accuracy': (y_test == y_predicted).sum() / total,
        }

graph = tf.Graph()
with graph.as_default():
    model = Model()
    objective = Objective(model, learning_rate=0.5)
    initialize = tf.global_variables_initializer()

with tf.Session(graph=graph) as session:
    session.run(initialize)
    experiment = Experiment()
    display(experiment.run(data, model, objective, session))